## Books Recommender System using CLustering | Collaborative Based Filtering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
books = pd.read_csv('data/old/Books.csv', encoding='latin-1')

C:\Users\Mukand\AppData\Local\Temp\ipykernel_13980\1181385418.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/old/Books.csv', encoding='latin-1')


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [6]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [7]:
books.rename(columns={"Book-Title":"Title",
                     "Book-Author": 'Author',
                     'Year-Of-Publication':'Year',
                     'Image-URL-L':'Img-URL'},
            inplace=True)

In [8]:
books.shape

(271360, 6)

In [9]:
users = pd.read_csv('data/old/Users.csv', encoding='latin-1')

In [10]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
users.rename(columns={'User-ID':'ID'}, inplace=True)

In [12]:
ratings = pd.read_csv('data/old/Ratings.csv', encoding='latin-1')

In [13]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [14]:
ratings.rename(columns={'User-ID':'ID', 'Book-Rating':'Rating'}, inplace=True)

shapes

In [15]:
ratings['ID'].value_counts()

ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
104999        1
105002        1
105008        1
105014        1
123969        1
Name: count, Length: 95513, dtype: int64

In [16]:
ratings['ID'].unique().shape

(95513,)

#### those users who has rated more than 300 books

In [16]:
x=ratings['ID'].value_counts()>200
x[x].shape

(815,)

In [17]:
y=x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
        88793,  33145, 116122,   9856,  73681,  28634,  59727, 188951, 155916,
        44296],
      dtype='int64', name='ID', length=815)

In [18]:
ratings = ratings[ratings['ID'].isin(y)]

In [19]:
ratings.shape

(482728, 3)

In [20]:
ratings_with_books = ratings.merge(books, on="ISBN")

In [21]:
ratings_with_books.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [22]:
ratings_with_books.shape

(446867, 8)

In [23]:
num_rating = ratings_with_books.groupby('Title')['Rating'].count().reset_index()

In [26]:
num_rating.head(13)

,Title,Rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
5,Dark Justice,1
6,Deceived,1
7,Earth Prayers From around the World: 365 Pray...,3
8,Final Fantasy Anthology: Official Strategy Gu...,3
9,Flight of Fancy: American Heiresses (Zebra Ba...,1


In [27]:
num_rating.rename(columns={'Rating':'Num_of_rating'}, inplace=True)

In [28]:
num_rating

,Title,Num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
153236,Ã?Ã?ber das Fernsehen.,2
153237,Ã?Ã?ber die Pflicht zum Ungehorsam gegen den...,3
153238,Ã?Ã?lpiraten.,1
153239,Ã?Ã?rger mit Produkt X. Roman.,1


In [29]:
final_rating = ratings_with_books.merge(num_rating, on ='Title')

In [30]:
final_rating.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL,Num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74


In [31]:
final_rating.shape

(446867, 9)

#### book which got more than 70 ratings

In [40]:
final_rating = final_rating[final_rating['Num_of_rating']>=65]

In [41]:
final_rating.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL,Num_of_rating
222,277427,60930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,123
223,254,60930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,123
224,6251,60930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,123
225,7346,60930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,123
226,11676,60930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,123


In [43]:
final_rating.shape

(25967, 9)

In [44]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='ID', index='Title', values= 'Rating')

In [45]:
book_pivot

ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,249628,249862,249894,250184,250405,250764,277427,277478,277639,278418
Title,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"A Child Called \It\"": One Child's Courage to Survive""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
White Oleander : A Novel,0.0,NaN,NaN,7.0,0.0,NaN,0.0,NaN,NaN,8.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
White Oleander : A Novel (Oprah's Book Club),NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
Wicked: The Life and Times of the Wicked Witch of the West,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,7.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
book_pivot.fillna(0, inplace=True)

### Training Model

In [47]:
from scipy.sparse import csr_matrix

In [48]:
book_sparse = csr_matrix(book_pivot)

In [49]:
book_sparse

<224x799 sparse matrix of type '<class 'numpy.float64'>'
	with 6652 stored elements in Compressed Sparse Row format>

In [50]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [51]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [52]:
distance, suggestion = model.kneighbors(book_pivot.iloc[13,:].values.reshape(1,-1), n_neighbors=6 )

In [53]:
distance

array([[ 0.        , 42.06839669, 42.16930163, 43.14220671, 43.31281566,
        43.41946568]])

In [54]:
suggestion

array([[ 13,  79, 153, 213, 112,  74]], dtype=int64)

In [55]:
book_pivot.iloc[12,:]

ID
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
250764    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: All I Really Need to Know, Length: 799, dtype: float64

In [56]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Along Came a Spider (Alex Cross Novels)',
       'How Stella Got Her Groove Back', 'The Deep End of the Ocean',
       'We'll Meet Again', 'Pretend You Don't See Her', 'Hideaway'],
      dtype='object', name='Title')


In [57]:
book_pivot.index[5]

'A Map of the World'

In [58]:
book_names = book_pivot.index

In [59]:
book_names[2]

'A Bend in the Road'

In [60]:
np.where(book_pivot.index == 'The Rainmaker'
)[0][0]

188

### finding url

In [61]:
ids = np.where(final_rating['Title'] == "The Bridges of Madison County")[0][0]

In [62]:
final_rating.iloc[ids]['Img-URL']

'http://images.amazon.com/images/P/044651652X.01.LZZZZZZZ.jpg'

In [63]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])

In [64]:
book_name[0]

Index(['Along Came a Spider (Alex Cross Novels)',
       'How Stella Got Her Groove Back', 'The Deep End of the Ocean',
       'We'll Meet Again', 'Pretend You Don't See Her', 'Hideaway'],
      dtype='object', name='Title')

In [65]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['Title'] == name)[0][0]
    ids_index.append(ids)

In [66]:
for idx in ids_index:
    url = final_rating.iloc[idx]['Img-URL']
    print(url)

http://images.amazon.com/images/P/0446364193.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0451197410.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0451186923.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0671004565.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0671867156.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0590313088.01.LZZZZZZZ.jpg


In [67]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

### Testing model

In [68]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [69]:
book_name = "The Bridges of Madison County"
recommend_book(book_name)

You searched 'The Bridges of Madison County'

The suggestion books are: 

Isle of Dogs
Songs in Ordinary Time (Oprah's Book Club (Paperback))
Saving Faith
How Stella Got Her Groove Back
The Last Precinct


### SVD, or Singular Value Decomposition, algorithm used for collaborative filtering based on matrix factorization. It decomposes the user-item rating matrix into two smaller matrices:

User latent factors: These represent "underlying preferences" or hidden characteristics of users.
Item latent factors: These represent "intrinsic features" or characteristics of items. When multiplied together, these two matrices approximate the original rating matrix.